In [170]:
import numpy as np

In [171]:
def invert_changed_row(A_inv, x, i):
    l = A_inv.dot(x)
    if l[i] == 0:
        print('not invertable')
        return
    l_i = l[i]
    l[i] = -1
    l = - l / l_i

    n = A_inv.shape[0]
    E = np.eye(n)
    E[:, i] = np.array(l)

    
    A_hat = np.zeros((n, n))
    for k in range(n):
        for j in range(n):
            if k == i:
                A_hat[k, j] = E[k,k] * A_inv[k,j]
                continue
            A_hat[k, j] = E[k,k] * A_inv[k,j] + E[k, i] * A_inv[i, j]
    return A_hat

In [493]:
def simplex_method(c: np.array, x: np.array, B: np.array, A: np.array, b: np.array, verbose: int=False):

    B -= 1
    iter_cnt = 1
    
    if verbose == 2:
        print('\t\tSTARTING SIMPLEX METHOD')
        print('x:\n', x)
        print('B:\n', B+1)

    
    while True:
        A_b = A[:, B]
        if verbose == 2:
            print('\t\tITERATION: ', iter_cnt)
            print('A_b:\n', A_b)
            
        if iter_cnt == 1:
            A_b_inv = np.linalg.inv(A_b)
        else:
            A_b_inv = invert_changed_row(A_b_inv.copy(), A[:, j_0].copy(), k)
        if verbose == 2:
            print('A_b_inv:\n', A_b_inv)
            
        c_b = c[B]
        if verbose == 2:
            print('c:\n', c)
            print('c_b:\n', c_b)

        # potential vector
        u = c_b @ A_b_inv
        if verbose == 2:
            print('u:\n', u)

        # eveluation_vector
        nabla = u @ A - c
        if verbose == 2:
             print('nablas:\n', nabla)
        if (nabla >= -1e-8).all():
            if verbose == 1:
                print('\nmax c: ', c[c!=0] @ x[c!=0])
            return x, B
        
        # first neg element index, z, teta, teta_0
        j_0 = nabla.argmin()
        z = A_b_inv @ A[:,j_0]
        teta = np.array([x[B[j]] / el if el > 0 else float('inf') for j, el in enumerate(z)])
        teta_0 = teta.min()
        if verbose == 2:
            print('j_0:\n', j_0)
            print('z:\n', z)
            print('teta:\n', teta)
            print('teta_0:\n', teta_0)
            
        if teta_0 == float('inf'):
            return "Not Bounded"
        
        k = teta.argmin()
        if verbose == 2:
            print('k:\n', k)
            
        j_star = B[k]
        
        old_B = B.copy()
        B[k] = j_0

        # update current plan
        old_x = x.copy()
        for b_ind, b in enumerate(B):
            x[b] = x[b] - teta_0 * z[b_ind]

        x[j_0] = teta_0
        x[j_star] = 0
        
        if verbose == 2:
            print('New B:\n', B+1)
            print('New x:\n', x)
                    
        iter_cnt += 1

In [494]:
c = np.array([
    1, 1, 0, 0, 0
], dtype=float)

A = np.array([
    [-1, 1, 1, 0, 0],
    [1, 0, 0, 1, 0],
    [0, 1, 0, 0, 1]
], dtype=float)

b = np.array([
    1, 3, 2
], dtype=float)

x_start = np.array([
    0, 0, 1, 3, 2
], dtype=float)

B_start = np.array([
    3, 4, 5
])

simplex_method(c.copy(), x_start.copy(), B_start.copy(), A.copy(), b.copy(), False)

(array([3., 2., 2., 0., 0.]), array([2, 0, 1]))

In [495]:
c = np.array([
    3, 2, 0, 0, 0
], dtype=float)

A = np.array([
    [2, 1, 1, 0, 0],
    [2, 3, 0, 1, 0],
    [3, 1, 0, 0, 1]
], dtype=float)

b = np.array([
    18, 42, 24
], dtype=float)

B_start = np.array([
    3, 4, 5
], dtype=int)

x_start = np.array([
    0, 0, 18, 42, 24
], dtype=float)

  
simplex_method(c.copy(), x_start.copy(), B_start.copy(), A.copy(), b.copy(), False)

(array([ 3., 12.,  0.,  0.,  3.]), array([1, 4, 0]))

### First stage of simplex method

In [496]:
def first_stage_simplex(A: np.array, b: np.array, verbose: int=False):
    m = A.shape[0]
    n = A.shape[1]
    if verbose == 2:
        print('m*n:', m, 'x', n)
        print('b:\n', b)
    # make b not negative
    for idx, val in enumerate(b):
        if val < 0:
            A[idx, :] *= -1
            b[idx] *= -1
    
    c_hat = np.ones(m+n) * -1
    c_hat[np.arange(n)] = 0
    A_hat = np.hstack([A, np.eye(m)])
    if verbose == 2:
        print('c_hat:\n', c_hat)
        print('A_hat:\n', A_hat)
    
    # basic allowable plan
    x_hat = np.zeros(n+m)
    x_hat[np.arange(n, n+m)] = b
    B = np.arange(n, n+m)
    if verbose == 2:
        print('x_hat:\n', x_hat)
        print('B:\n', B+1)
    
    
    x, B = simplex_method(c_hat.copy(), x_hat.copy(), B.copy() + 1, A_hat.copy(), b.copy(), verbose=0)
    if verbose == 2:
        print('x:\n', x)
        print('B:\n', B)
    if (abs(x[np.arange(n, m+n)]) > 1e-8).all():
        print('Not compatible')
        return 'Not copatible'
    
    flag = True
    while flag:
        n, m = A.shape
        if (B < n).all():
            return A, b


        # find max index of free varibles
        j_k = 0
        k = 0
        i = 0
        for idx, val in enumerate(B):
            if val >= n:
                if j_k < val:
                    j_k = val
                    k = idx
                    i = n+m - j_k
        if verbose == 2:
            print('j_k:', j_k)
            print('k:', k)
            print('i:', i)

        newbies = set(np.arange(0, n)) - set(B)
        if verbose == 2:
            print('newbies:\n', newbies)
    
        l = []
        A_hat_b_inv = np.linalg.pinv(A_hat[:, B])
        if verbose == 2:
            print('A_hat_b:\n', A_hat[:, B])
            print('A_hat_b_inv:\n', A_hat_b_inv)

        repeat_cnt = 0
        for idx, each in enumerate(newbies):
            l.append(A_hat_b_inv @ A[:, each][:, np.newaxis])
            if abs(l[idx][k]) < 1e-8:
                repeat_cnt += 1
            else:
                B[k] = each


        if verbose == 2:
            print('l:\n', l)

        # deleting extras
        if repeat_cnt == len(l):
            A = np.delete(A, i, 0)
            A_hat = np.delete(A_hat, i, 0)
            b = np.delete(b, i, 0)
            B = np.delete(B, k, 0)
        else:
            flag = False
    return A, b
    

In [287]:
A = np.array([
    [1, 1, 1, 1, 1],
    [2, 2, 2, 2, 2],
    [3, 3, 3, 3, 3],
    [4, 4, 4, 4, 4]
    

], dtype=float)

b = np.array([0, 0, 0, 0], dtype=float)

answer = first_stage_simplex(A, b, verbose=2)
print()

if type(answer) is not tuple:
    print('Not compatible')
else:
    print('Have valid plan:')
    print('A: ', answer[0])
    print('b: ', answer[1])

m*n: 4 x 5
b:
 [0. 0. 0. 0.]
c_hat:
 [ 0.  0.  0.  0.  0. -1. -1. -1. -1.]
A_hat:
 [[1. 1. 1. 1. 1. 1. 0. 0. 0.]
 [2. 2. 2. 2. 2. 0. 1. 0. 0.]
 [3. 3. 3. 3. 3. 0. 0. 1. 0.]
 [4. 4. 4. 4. 4. 0. 0. 0. 1.]]
x_hat:
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
B:
 [6 7 8 9]
x:
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
B:
 [0 6 7 8]
j_k: 8
k: 3
i: 1
newbies:
 {1, 2, 3}
A_hat_b:
 [[1. 0. 0. 0.]
 [2. 1. 0. 0.]
 [3. 0. 1. 0.]
 [4. 0. 0. 1.]]
A_hat_b_inv:
 [[ 1.00000000e+00  2.22044605e-16  2.77555756e-17 -2.22044605e-16]
 [-2.00000000e+00  1.00000000e+00  2.22044605e-16  2.77555756e-16]
 [-3.00000000e+00 -9.43689571e-16  1.00000000e+00  7.28583860e-16]
 [-4.00000000e+00 -1.27675648e-15 -2.42861287e-16  1.00000000e+00]]
l:
 [array([[1.00000000e+00],
       [1.11022302e-15],
       [1.77635684e-15],
       [8.88178420e-16]]), array([[1.00000000e+00],
       [1.11022302e-15],
       [1.77635684e-15],
       [8.88178420e-16]]), array([[1.00000000e+00],
       [1.11022302e-15],
       [1.77635684e-15],
       [8.88178420e-16]

### Dual simplex method

In [497]:
def dual_simplex(c: np.array, A: np.array, b:np.array, B: np.array, verbose=0):
    m, n = A.shape[0], A.shape[1]
    B -= 1
    iter = 1
    
    while True:
        if verbose:
            print('\tITERATION', iter)
        A_b = A[:, B]
        A_b_inv = np.linalg.inv(A_b)
        c_b = c[B]
        y = c_b @ A_b_inv
        k = np.zeros(n)
        k_b = A_b_inv @ b
        k[B] = k_b
        if verbose:
            print('k: ', k)
        j_k = None
        k_idx = -1
        for idx, val in enumerate(k):
            if val < 0:
                j_k = idx
                k_idx += 1
            if (j_k is None) and (idx == len(k)-1):
                return k, B
        
        j = set(range(n)) - set(B)
        if verbose:
            print('j: ', j)

        y_delta = A_b_inv[k_idx]
        if verbose:
            print('y_delta: ', y_delta)
            
        mu = []
        for j_idx in j:
            mu_ = y_delta @ A[:, j_idx]
            mu.append(mu_)
        if verbose:
            print('mu: ', mu)
            
        if (np.array(mu) > 0).all():
            return 'Not compatible :('
        sigma = []
        for idx, j_idx in enumerate(j):
            sigma_ = (c[j_idx] - (A[:, j_idx] @ y)) / mu[idx]
            sigma.append(sigma_)
        j_0 = np.argmin(sigma)
        if verbose:
            print('sigmas: ', sigma)
            print('j_0: ', j_0)
        B[k_idx] = j_0
        if verbose:
            print('B:\n', B)
        
        
        j_k = None
        k_idx = -1
        iter += 1
    

In [499]:
c = np.array([
    -4, -3, -7, 0, 0
])

A = np.array([
    [-2, -1, -4, 1, 0],
    [-2, -2, -2, 0, 1]
])

b = np.array([
    -1, -1.5
])

B_start = np.array([
    4, 5
])


answer = dual_simplex(c.copy(), A.copy(), b.copy(), B_start.copy(), verbose=False)
print()


if type(answer) is not tuple:
    print('Not compatible')
else:
    print('Have valid plan:')
    print('A: ', answer[0])
    print('b: ', answer[1])


Have valid plan:
A:  [0.25 0.5  0.   0.   0.  ]
b:  [0 1]


### Transport Task

In [491]:
def transport(a: np.array, b: np.array, c: np.array, verbose=True):
    m = c.shape[0]
    n = c.shape[1]
    iter_num = 1
    B = None
    
    while True:
        if verbose: 
            print('\t\tITERATION: ', iter_num)
            
        if iter_num == 1:
            plan = find_base_plan(a.copy(), b.copy())
            B = set()
            for i in range(m):
                for j in range(n):
                    if plan[i, j] > 0:
                        B.add((i, j))
            if verbose:
                print('First Base plan (northwest corner):\n', plan)
                print('current cost:', get_cost(plan, c))
        else:
            if verbose:
                print('Current Base plan:\n', plan)
                print('current cost:', get_cost(plan, c))

        mask = c.copy()
        for i in range(m):
            for j in range(n):
                if (i, j) not in B:
                    mask[i, j] = 0
        if verbose:
            print('mask:\n', mask)
            
        a_poten, b_poten = find_potentials(mask, B)
        print(B)
        if verbose:
            print('a_poten: ', a_poten)
            print('b_poten: ', b_poten)

        eps = 1e-8
        evaluation_plan = np.array((abs(mask) < eps), dtype=float)
        evaluation_plan[evaluation_plan == 0] = np.inf

        evaluation_plan, is_optim, max_positive, i_idx, j_idx = find_nums_not_on_path(a_poten.copy(), b_poten.copy(), evaluation_plan.copy(), c.copy())

        if verbose:
            print(f'eval_plan:\n', evaluation_plan)
            print('is optimal: ', is_optim)
        if is_optim:
            return plan, B, get_cost(plan, c)

        plus_minus = fill_plus_minus(np.array(mask > 0, dtype='int'), i_idx, j_idx)
        if verbose:
            print('plus_minus:\n', plus_minus)
        
        plan, B = recalc(plan.copy(), plus_minus.copy())
        if verbose:
            print('new base_plan:\n', plan)
            print()
        
        iter_num += 1
        

def recalc(base_plan, plus_minus):
    m, n = base_plan.shape[0], base_plan.shape[1]
    
    B = set()
    for i in range(m):
        for j in range(n):
            if base_plan[i, j] > 0:
                B.add((i, j))
                
    inds = np.where(plus_minus == -1)
    min_ = np.inf
    i_min_, j_min_ = -1, -1
    pos = 0
    for i, j in zip(inds[0], inds[1]):
        if min_ > base_plan[i, j]:
            pos += 1
            min_ = base_plan[i, j]
            i_min_, j_min_ = i, j
    
    # decrease negatives
    if pos == 2:
        base_plan[inds[0][0], inds[1][0]] -= min_
        base_plan[i_min_, j_min_] = 0
        B.remove((i_min_, j_min_))
    elif pos == 1:
        base_plan[inds[0][1], inds[1][1]] -= min_
        base_plan[i_min_, j_min_] = 0
        B.remove((i_min_, j_min_))

    
    
    # increase posives
    inds_pos = np.where(plus_minus == 1)
    for i, j in zip(inds_pos[0], inds_pos[1]):
        base_plan[i, j] += min_
        B.add((i, j))
    
    return base_plan, B
    
    

def fill_plus_minus(mask, i_idx, j_idx):
    def check_horizontal(i, j):
        i_s = set(range(mask.shape[0])) - set([i_idx])
        for i_ in i_s:
            if mask[i_, j] == 1:
                if mask[i_, :].sum() > 1:
                    return i_
            
    def check_vertical(i, j):
        j_s = set(range(mask.shape[1])) - set([j_idx])
        for j_ in j_s:
            if mask[i, j_] == 1:
                if mask[:, j_].sum() > 1:
                    return j_
        
    plus_minus = np.empty((mask.shape[0], mask.shape[1]))
    plus_minus[:] = np.NaN
    
    # set first plus
    plus_minus[i_idx, j_idx] = 1
    
    i_idx_1, j_idx_2 = i_idx, j_idx
    
    for i in range(1, 4):
        if i == 3:
            plus_minus[i_idx_1, j_idx] = -1
            return plus_minus 
        if i % 2 == 1:
            # make minus
            i_idx = check_horizontal(i_idx, j_idx)
            plus_minus[i_idx, j_idx] = -1

            continue
        if i % 2 == 0:
            # make plus
            j_idx = check_vertical(i_idx, j_idx)
            plus_minus[i_idx, j_idx] = 1
            continue
    return plus_minus
            
    
    
def find_nums_not_on_path(a_poten, b_poten, eval_plan, c):
    is_optim = True
    max_positive = -np.inf
    i_idx, j_idx = None, None
    for i in range(c.shape[0]):
        for j in range(c.shape[1]):
            if eval_plan[i, j] != np.inf:
                eval_plan[i, j] = b_poten[j] - a_poten[i] - c[i, j]
                if eval_plan[i, j] > 0:
                    is_optim = False
                    if max_positive < eval_plan[i, j]:
                        max_positive = eval_plan[i, j]
                        i_idx, j_idx = i, j
    return eval_plan, is_optim, max_positive, i_idx, j_idx
    
    

def get_cost(plan, c):
    return (plan * c).sum()


def find_potentials(c, B):
    m = c.shape[0]
    n = c.shape[1]
    A = np.zeros((m+n, m+n), dtype=int)
    b = np.zeros((m+n), dtype=int)
    b = []
    curr = 0
    for i in range(m):
        for j in range(n):
            if (i, j) in B:
                A[curr, i+3] = -1
                A[curr, j] = 1
                b.append(c[i, j])
                curr += 1
    A[-1, 3] = -1
    b.append(0)
    x = np.array(np.linalg.solve(A, b), dtype=int)
    b_poten = x[:n]
    a_poten = x[n:]
    return a_poten, b_poten
    
    
def find_base_plan(a, b):
    plan = np.zeros((len(a), len(b)))
    i, j = 0, 0
    while (a.sum() > 0) and (b.sum() > 0):
        plan[i, j] = min(a[i], b[j])
        if a[i] > b[j]:
            a[i] -= b[j]
            b[j] = 0
            j += 1
        else:
            b[j] -= a[i]
            a[j] = 0
            i += 1
    return plan
            
            

In [500]:
a = np.array([
    100, 300, 300
])

b = np.array([
    300, 200, 200
])

c = np.array([
    [8, 4, 1],
    [8, 4, 3],
    [9, 7, 5]
])

plan, B, cost = transport(a.copy(), b.copy(), c.copy())
print()
print('Optimal plan:')
print(plan)
print('B: ', B)
print('Min cost: ', cost)

		ITERATION:  1
First Base plan (northwest corner):
 [[100.   0.   0.]
 [200. 100.   0.]
 [  0. 100. 200.]]
current cost: 4500.0
mask:
 [[8 0 0]
 [8 4 0]
 [0 7 5]]
{(2, 1), (0, 0), (1, 1), (2, 2), (1, 0)}
a_poten:  [ 0  0 -3]
b_poten:  [8 4 2]
eval_plan:
 [[inf  0.  1.]
 [inf inf -1.]
 [ 2. inf inf]]
is optimal:  False
plus_minus:
 [[nan nan nan]
 [-1.  1. nan]
 [ 1. -1. nan]]
new base_plan:
 [[100.   0.   0.]
 [100. 200.   0.]
 [100.   0. 200.]]

		ITERATION:  2
Current Base plan:
 [[100.   0.   0.]
 [100. 200.   0.]
 [100.   0. 200.]]
current cost: 4300.0
mask:
 [[8 0 0]
 [8 4 0]
 [9 0 5]]
{(0, 0), (1, 1), (2, 0), (2, 2), (1, 0)}
a_poten:  [ 0  0 -1]
b_poten:  [8 4 4]
eval_plan:
 [[inf  0.  3.]
 [inf inf  1.]
 [inf -2. inf]]
is optimal:  False
plus_minus:
 [[-1. nan  1.]
 [nan nan nan]
 [ 1. nan -1.]]
new base_plan:
 [[  0.   0. 100.]
 [100. 200.   0.]
 [200.   0. 100.]]

		ITERATION:  3
Current Base plan:
 [[  0.   0. 100.]
 [100. 200.   0.]
 [200.   0. 100.]]
current cost: 4000.0
m

In [444]:
def find_poten(c):
    m = c.shape[0]
    n = c.shape[1]
    A = np.zeros((m+n, m+n), dtype=int)
    b = np.zeros((m+n), dtype=int)
    b = []
    curr = 0
    for i in range(m):
        for j in range(n):
            if c[i, j] > 0:
                A[curr, i+3] = -1
                A[curr, j] = 1
                b.append(c[i, j])
                curr += 1
    A[-1, 3] = -1
    b.append(0)
    solve = np.array(np.linalg.solve(A, b), dtype=int)
    b_poten = solve[:n]
    a_poten = solve[n:]
    return a_poten, b_poten
    
    
a = np.array([
    [8, 0, 0],
    [8, 4, 0],
    [0, 7, 5]
])

a_2 = np.array([
    [8, 0, 0],
    [8, 4, 0],
    [9, 0, 5]
], dtype='int')

find_poten(a_2)

A:
[[ 1  0  0 -1  0  0]
 [ 1  0  0  0 -1  0]
 [ 0  1  0  0 -1  0]
 [ 1  0  0  0  0 -1]
 [ 0  0  1  0  0 -1]
 [ 0  0  0 -1  0  0]]
b: 
[8, 8, 4, 9, 5, 0]
x:
[ 8  4  4  0  0 -1]
a:  [ 0  0 -1]
b:  [8 4 4]
